In [ ]:
import sys
import time
import torch
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader

from datasets.dataset import SMPLRandomDataset
from models.SkinningNet import WeightBindingNet
from utils.LossFunction import WeightBindingLoss

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:%s"%device)

In [ ]:
batchSize = 16
trainDataset = SMPLRandomDataset(complexity = "skinning", 
                                 gender = "mixed", 
                                 dataSize = 5000, 
                                 vertexOrderRandom = True, 
                                 noise = 1, 
                                 rotate = False, 
                                 scale = True,
                                 translate = True,
                                 centre = False)
trainIter = DataLoader(trainDataset, num_workers=0, batch_size=batchSize, shuffle=True, drop_last=True)
allTrainNUM = len(trainDataset)

In [ ]:
net = WeightBindingNet(jointNum = 24)
net = net.to(device)

In [ ]:
lr = 0.0001
epochs =100
opt = torch.optim.Adam(net.parameters(), lr=lr)
scheduler = CosineAnnealingLR(opt, epochs, eta_min=0.1*lr)

loss = WeightBindingLoss()

In [ ]:
beginTime = time.time()
net = net.train()

for epoch in range(epochs):
    epochBegin = time.time()
    trainLoss = 0.0
    completedNum = 0
    batchNum = 0
    trainAcc = 0.0
    for V, facesOneRingIdx, rigW, joints in trainIter:
        V = V.to(device).float()
        facesOneRingIdx = facesOneRingIdx.to(device).long()
        rigW = rigW.to(device).float()
        
        opt.zero_grad()
        
        preA = net(V, facesOneRingIdx)
        l, acc = loss(preA, rigW.permute(0, 2, 1))
        l.backward()
        opt.step()
        
        batchLoss = float(l)
        batchAcc = float(acc)
        trainAcc += batchAcc
        trainLoss += batchLoss
        batchNum += 1
        completedNum += batchSize
        compltePer = completedNum/allTrainNUM
        leftTime = ((time.time() - epochBegin)/compltePer - (time.time() - epochBegin))/60
        sys.stdout.write('\r Trainning %i / %i, weight loss: %f, acc: %0.2f%%, percentage: %0.2f%%, remain %d minuetes'%(completedNum, allTrainNUM, batchLoss, batchAcc*100, compltePer * 100, leftTime))
        sys.stdout.flush()
    
    save_path = './results/WeightBindingNet/'
    if epoch%5 == 0:
        torch.save(net.state_dict(), save_path + "%d.pkl"%epoch)
    torch.save(net.state_dict(), save_path + "latest.pkl")
    print("\n")
    print("Epoch%d, weight loss: %f, acc: %0.2f%%, time cost: %0.2f minuetes"%(epoch+1, trainLoss/batchNum, trainAcc/batchNum*100, (time.time() - beginTime) / 60 ))
    print("----------------------------------")
    scheduler.step()